In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date,col, unix_timestamp

In [4]:
spark = SparkSession.builder.master('local[*]').appName("Module4HW").getOrCreate()
print(f'spark version: {spark.version}')

spark version: 3.5.4


In [3]:
!wget  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-02 16:26:06--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.165.247.187, 3.165.247.72, 3.165.247.163, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.165.247.187|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  1.15MB/s    in 53s     

2025-03-02 16:27:03 (1.16 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [7]:
df = spark.read.parquet("yellow_tripdata_2024-10.parquet")
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [9]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [10]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [11]:
df_repart  = df.repartition(4)
df_repart.write.mode('overwrite').parquet("yellow_tripdata_2024-10_repart.parquet")

In [ ]:
import os

# Directory where the repartitioned parquet files are stored
parquet_dir = "yellow_tripdata_2024-10_repart.parquet"

# List all files in the directory that end with .parquet (ignoring _SUCCESS file, if any)
file_paths = [
    os.path.join(parquet_dir, file)
    for file in os.listdir(parquet_dir)
    if file.endswith(".parquet")
]

# Get sizes in bytes and compute average size
sizes_bytes = [os.path.getsize(file_path) for file_path in file_paths]
print(f'len size_bytes : {len(sizes_bytes)}')
print(f'size_bytes : {sizes_bytes}')
average_size_bytes = sum(sizes_bytes) / len(sizes_bytes)
average_size_mb = average_size_bytes / (1024 * 1024)

print(f"Average Parquet file size: {average_size_mb:.2f} MB")


len size_bytes : 4
size_bytes : [24168482, 24143003, 24180654, 24154000]
Average Parquet file size: 23.04 MB


In [20]:
df_october_15 = df.filter(
    to_date(col("tpep_pickup_datetime")) == "2024-10-15"
)

count_oct15 = df_october_15.count()
print(f"Number of trips on October 15: {count_oct15}")

Number of trips on October 15: 128893


In [21]:
df_duration = df.withColumn(
    'trip_duration_hours',
    (unix_timestamp('tpep_dropoff_datetime') - unix_timestamp('tpep_pickup_datetime')) / 3600
)

max_trip_duration = df_duration.agg(
    {"trip_duration_hours": "max"}
).collect()[0][0]

print(f'longest trip duration: {max_trip_duration:.2f} hours')



longest trip duration: 162.62 hours


In [22]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-02 17:14:17--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.165.247.79, 3.165.247.72, 3.165.247.187, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.165.247.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.008s  

2025-03-02 17:14:18 (1.41 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [23]:
zone_df = spark.read.option('header', 'true').csv("taxi_zone_lookup.csv")
zone_df.createOrReplaceTempView("taxi_zones")
df.createOrReplaceTempView("yellow_trips")

In [24]:
zone_df.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [25]:
zone_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [28]:
# compute pickup counts per zone
least_frequent = spark.sql('''
    SELECT z.zone, COUNT(*) AS pickup_count
    FROM yellow_trips y
    JOIN taxi_zones z
    ON y.PULocationID = z.LocationID
    GROUP BY z.zone
    ORDER BY pickup_count
    LIMIT 1
''')

least_frequent.show()

+--------------------+------------+
|                zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
+--------------------+------------+

